In [14]:
import sys
import json
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 
from tqdm.notebook import tqdm

In [15]:
queries_dir = Path('queries/')
enum_query = read_sql(queries_dir / "1.enum_query.sql")
table_query = read_sql(queries_dir / "3.table_query.sql")

datasets_dir = Path("../datasets/")
haccp_dataset_path = datasets_dir / "haccp_dataset_200619152008.csv"
veg_rawmtrl_path = datasets_dir / "VegRawMtrl_200620104858.csv"
all_rawmtrl_path = datasets_dir / "AllergyRawMtrl_200620104858.csv"
# veg_prdkind_path = datasets_dir / "VegPrdKind_200619145823.csv"
# all_prdkind_path = datasets_dir / "AllergyPrdKind_200619145823.csv"


In [16]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
haccp_dataset = haccp_dataset.drop(['rnum', 'productgb', 'prdkindstate'], axis=1)

In [17]:
if sys.platform.lower() == 'darwin':
    conn = connect(user='postgres', password='0000', db='projectDB', host='localhost', port=5433)
elif sys.platform == 'win32':
    conn = connect(user='postgres', password='1234', db='projectDB', host='localhost', port=5432)

In [18]:
initializ_db(conn)

In [19]:
execute_queries(conn, enum_query)
execute_queries(conn, table_query)

In [20]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
haccp_dataset = haccp_dataset.drop(['rnum', 'productgb', 'prdkindstate'], axis=1)
    
veg_rawmtrl = pd.read_csv(veg_rawmtrl_path)
all_rawmtrl = pd.read_csv(all_rawmtrl_path)
# veg_prdkind = pd.read_csv(veg_prdkind_path)
# all_prdkind = pd.read_csv(all_prdkind_path)

In [21]:
replace_dict = {
    '(' : ' ',
    ')' : ' ',
    '%' : '%%',
    "'" : '',
    '\n' : ' ',
    '\r' : ' '
}

In [22]:
for data in tqdm(haccp_dataset.values):
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    reportno, prdnm, rawmtrl, allergy, nutrient, barcode, prdkind, manu, seller, capa, img1, img2 = new_data
    try: 
        barcode = int(barcode)
    except ValueError:
        barcode = 0
    val = "{reportno}, '{prdnm}', '{rawmtrl}', '{allergy}', '{nutrient}', {barcode},\
    '{prdkind}', '{manu}', '{seller}', '{capa}', '{img1}', '{img2}'".format(reportno = int(reportno), prdnm = prdnm, rawmtrl = rawmtrl,
                                                                 allergy = allergy, nutrient = nutrient, barcode = int(barcode),
                                                                 prdkind = prdkind, manu=manu, seller=seller, capa=capa, img1=img1, img2=img2)
    insert_query = "INSERT INTO foodinfo VALUES({})".format(val)
    execute_query(conn, insert_query)

In [23]:
for data in tqdm(veg_rawmtrl.values):
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    rawMtrl, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg = new_data
    val = "'{}', '{}', '{}', '{}',' {}', '{}', '{}'".format(rawMtrl, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg)
    insert_query = "INSERT INTO VegRawMtrl VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

In [24]:
for data in tqdm(all_rawmtrl.values):
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    rawMtrl, allergy, isCrossReact, parentAllergy, probablility = new_data
    val = "'{}', '{}', '{}', '{}', '{}'".format(rawMtrl, allergy, isCrossReact, parentAllergy, probablility)
    insert_query = "INSERT INTO AllergyRawMtrl VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

---

In [25]:
for data in tqdm(veg_prdkind.values):
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    prdKind, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg = new_data
    val = "'{}', '{}', '{}', '{}',' {}', '{}', '{}'".format(prdKind, vegan, lactoVeg, ovoVeg, lactoOvoVeg, pescoVeg, polloVeg)
    insert_query = "INSERT INTO VegPrdKind VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

NameError: name 'veg_prdkind' is not defined

In [26]:
for data in tqdm(all_prdkind.values):
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    prdKind, allergy, isCrossReact, parentAllergy, probablility = new_data
    val = "'{}', '{}', '{}', '{}'".format(prdKind, allergy, isCrossReact, parentAllergy, probablility)
    insert_query = "INSERT INTO AllergyPrdKind VALUES({})".format(val)
    execute_query(conn, insert_query, log=False)

NameError: name 'all_prdkind' is not defined